# TileDB 101: Vector Search

In this notebook we provide a quickstart tutorial on the vector search capabilities of TileDB. I strongly recommend you read the blog "[Why TileDB for Vector Search](https://tiledb.com/blog/why-tiledb-as-a-vector-database)" before digging into this tutorial, especially if you are not familiar with the TileDB array database and how it naturally _morphs_ into a vector database (vectors are 1D arrays after all). 

TileDB’s core array technology lies in the open-source (MIT License) library [TileDB-Embedded](https://github.com/TileDB-Inc/TileDB), but we developed the vector search specific components in library [TileDB-Vector-Search](https://github.com/TileDB-Inc/TileDB-Vector-Search), which is also open-source under MIT License. Similar to the core library, TileDB-Vector-Search is built in C++, but it also offers a Python API. 

In the majority of this example, we will use the Python API of TileDB-Vector-Search, and the examples are reproducible on your local machine. However, TileDB also develops the commercial [TileDB Cloud](https://tiledb.com/products/tiledb-cloud/) product, which provides you with additional governance and scalability features, which we cover briefly in one section. You can download this notebook and run it locally, or launch a Jupyter server directly in TileDB Cloud. [Sign up](https://cloud.tiledb.com/auth/signup) to do so (no credit card required), and we’ll give you **free credits** so that you can evaluate without a hassle.

# Setup

In general, install TileDB-Vector-Search locally like so:

```bash
# Using pip
pip install tiledb-vector-search

# Or, using conda (requires tiledb-cloud and scikit-learn)
pip install tiledb-cloud
conda install -c conda-forge -c tiledb tiledb-vector-search
```

To run this tutorial locally, we used the following package versions:

```bash
conda install -c conda-forge -c tiledb tiledb-vector-search==0.0.15 tiledb==2.17.0 python=3.9.18
pip install tiledb-cloud==0.10.21
```

We’ll first work with the small (10k) SIFT dataset from the [Datasets for approximate nearest neighbor search](http://corpus-texmex.irisa.fr/) site. Download the dataset from [here]((https://github.com/TileDB-Inc/TileDB-Vector-Search/releases/download/0.0.1/siftsmall.tgz) \(mirrored, from original source [here](ftp://ftp.irisa.fr/local/texmex/corpus/sift.tar.gz)\). The code below will extract the tarball automatically, but if needed run the following to extract:

```
tar xf siftsmall.tgz
```

Later we’ll play with the 1B SIFT dataset on TileDB Cloud, which we have pre-ingested for you, so nothing to do here.

# A Simple Vector Search Example

In this example I will show you how to ingest the small dataset you just downloaded, and run your first similarity search query. For more information, see the [TileDB-Vector-Search API Reference](https://tiledb-inc.github.io/TileDB-Vector-Search/documentation/reference/).

We start by importing the necessary libraries:

In [1]:
import tiledb
import tiledb.vector_search as vs
from tiledb.vector_search.utils import *
import os, shutil
import urllib
import numpy as np

# Download data

This cell will automatically download and extract the SIFT 10k dataset from ftp://ftp.irisa.fr/local/texmex/corpus/sift.tar.gz (downloading from github mirror for reliability), and then extract the tarball:

In [2]:
data_uri = "https://github.com/TileDB-Inc/TileDB-Vector-Search/releases/download/0.0.1/siftsmall.tgz"
data_filename = "siftsmall.tar.gz"

def download_file(url, filename):
    import tarfile, os, urllib.request

    if not os.path.exists(os.path.dirname(filename)):
        os.makedirs(os.path.dirname(filename))

    if not os.path.exists(filename):
        print(f"Downloading {url} to {filename}")
        urllib.request.urlretrieve(url, filename)
    else:
        print("File %s already exists" % filename)

    tarfile.open(filename, "r:gz").extractall(os.path.dirname(filename))

data_dir = "/tmp/sift10k/"
local_data_path = os.path.join(data_dir, data_filename)

download_file(data_uri, local_data_path)

File /tmp/sift10k/siftsmall.tar.gz already exists


## Ingestion
You can ingest vectors with a single command as follows.

In [3]:
index_uri = os.path.expanduser("~/sift10k_flat")

if os.path.exists(index_uri):
    shutil.rmtree(index_uri)
    print("Deleted previous local index.")

flat_index = vs.ingest(
    index_type = "FLAT",
    index_uri = index_uri,
    source_uri = "/tmp/sift10k/siftsmall_base.fvecs",
    source_type = "FVEC",
    partitions=100
)

Deleted previous local index.


In [4]:
%%bash
ls -al ~/sift10k_flat # List the group contents

total 24
drwxr-sr-x  6 jovyan users 4096 Oct  4 10:04 .
drwxrwsr-x 20 root   users 4096 Oct  4 10:04 ..
drwxr-sr-x  2 jovyan users 4096 Oct  4 10:04 __group
drwxr-sr-x  2 jovyan users 4096 Oct  4 10:04 __meta
drwxr-sr-x  8 jovyan users 4096 Oct  4 10:04 shuffled_vector_ids_10_04_2023_10_04_49
drwxr-sr-x  8 jovyan users 4096 Oct  4 10:04 shuffled_vectors_10_04_2023_10_04_49
-rw-r--r--  1 jovyan users    0 Oct  4 10:04 __tiledb_group.tdb


In [5]:
group = tiledb.Group(index_uri, "r")
group

sift10k_flat GROUP
|-- shuffled_vector_ids_10_04_2023_10_04_49 ARRAY
|-- shuffled_vectors_10_04_2023_10_04_49 ARRAY

In [6]:
index = vs.FlatIndex(index_uri)

# Open the array
A = tiledb.open(index.db_uri)

# Print the schema - 2D dense array
print(A.schema)

ArraySchema(
  domain=Domain(*[
    Dim(name='rows', domain=(0, 127), tile=128, dtype='int32', filters=FilterList([ZstdFilter(level=-1), ])),
    Dim(name='cols', domain=(0, 9999), tile=100, dtype='int32', filters=FilterList([ZstdFilter(level=-1), ])),
  ]),
  attrs=[
    Attr(name='values', dtype='float32', var=False, nullable=False, enum_label=None, filters=FilterList([ZstdFilter(level=-1), ])),
  ],
  cell_order='col-major',
  tile_order='col-major',
  sparse=False,
)



In [7]:
# Print the first vector
print(A[:,0]["values"])

[  0.  16.  35.   5.  32.  31.  14.  10.  11.  78.  55.  10.  45.  83.
  11.   6.  14.  57. 102.  75.  20.   8.   3.   5.  67.  17.  19.  26.
   5.   0.   1.  22.  60.  26.   7.   1.  18.  22.  84.  53.  85. 119.
 119.   4.  24.  18.   7.   7.   1.  81. 106. 102.  72.  30.   6.   0.
   9.   1.   9. 119.  72.   1.   4.  33. 119.  29.   6.   1.   0.   1.
  14.  52. 119.  30.   3.   0.   0.  55.  92. 111.   2.   5.   4.   9.
  22.  89.  96.  14.   1.   0.   1.  82.  59.  16.  20.   5.  25.  14.
  11.   4.   0.   0.   1.  26.  47.  23.   4.   0.   0.   4.  38.  83.
  30.  14.   9.   4.   9.  17.  23.  41.   0.   0.   2.   8.  19.  25.
  23.   1.]


Parameter `partitions` dictates the tiling of this array, but you can ignore it for now (we’ll discuss it at length in a separate blog). This vector search asset has no indexing (`index_type = FLAT`). Therefore, running this ingestion function is rapid, similarity search (for large datasets) is slow as it is brute-force, and the recall (i.e., accuracy) is always 100%. 

To ingest the dataset building an `IVF_FLAT` index, all you need to do it specify `index_type = "IVF_FLAT"`:

In [8]:
index_uri_ivf = os.path.expanduser("~/sift10k_ivf_flat")

if os.path.exists(index_uri_ivf):
    shutil.rmtree(index_uri_ivf)
    print("Deleted previous local index.")

ivf_flat_index = vs.ingest(
    index_type="IVF_FLAT",
    source_uri="/tmp/sift10k/siftsmall_base.fvecs",
    index_uri=index_uri_ivf,
    source_type = "FVEC",
    partitions = 100
)

Deleted previous local index.


This takes a longer time to run, similarity search is much faster even for humongous datasets, but the recall may be smaller than 100%.

Looking into the contents of the created group `sift10k_ivf_flat`, we can now see that there are more arrays created (`centroids.tdb`, `ids.tdb` and `index.tdb` – the names may change in future versions), which collectively comprise the `IVF_FLAT` index.

In [9]:
%%bash
ls -l ~/sift10k_ivf_flat

total 24
drwxr-sr-x 2 jovyan users 4096 Oct  4 10:04 __group
drwxr-sr-x 2 jovyan users 4096 Oct  4 10:04 __meta
drwxr-sr-x 8 jovyan users 4096 Oct  4 10:04 partition_centroids_10_04_2023_10_04_50
drwxr-sr-x 8 jovyan users 4096 Oct  4 10:04 partition_indexes_10_04_2023_10_04_50
drwxr-sr-x 8 jovyan users 4096 Oct  4 10:04 shuffled_vector_ids_10_04_2023_10_04_50
drwxr-sr-x 8 jovyan users 4096 Oct  4 10:04 shuffled_vectors_10_04_2023_10_04_50
-rw-r--r-- 1 jovyan users    0 Oct  4 10:04 __tiledb_group.tdb


You can explore the schema of those arrays and read their contents as you’d do with any other TileDB array. We will cover the TileDB-Vector-Search internals in detail in future blogs. Our blog post [TileDB 101: Arrays](https://tiledb.com/blog/tiledb-101-arrays/) can familiarize you with TileDB arrays.

## Similarity Search

To run similarity search on the ingested vectors, we’ll load the queries and ground truth vectors from the `siftsmall` dataset we downloaded (noting though you can use any vector to query this dataset). We provide load\_fvecs and load\_ivecs as auxiliary functions in the `tiledb.vector_search.utils` module.

In [10]:
# Get query vectors with ground truth
query_vectors = load_fvecs("/tmp/sift10k/siftsmall_query.fvecs")
ground_truth = load_ivecs("/tmp/sift10k/siftsmall_groundtruth.ivecs")

To return the most similar vectors to a query vector, simply run:

In [11]:
# Select a query vector
query_id = 77
qv = np.array([query_vectors[query_id]])

# Return the 100 most similar vectors to the query vector with FLAT
result_d, result_i = flat_index.query(qv, k=100)

# Return the 100 most similar vectors to the query vector with IVF_FLAT
# (you can set the nprobe parameter)
#result_d, result_i = ivf_flat_index.query(qv, nprobe = 10, k=100)
 
result_i

array([[8578, 8275, 5332, 9153, 2092, 3290, 2010, 8004, 2949, 9190, 6784,
        5756, 1721, 1075, 1411, 6994, 4120, 7529, 3994, 6878, 4844, 3969,
        3313, 7945,  796, 2135, 2861, 5957,  621,  905, 5014, 9429, 7287,
        4675, 9172, 2399, 1923,  902, 7555, 2986, 1976,  852, 2834, 5247,
        4545, 4231, 3431, 4614, 8526, 4045, 2987, 4602,   23, 7018, 1555,
        4637, 2156, 6058, 7331, 9181, 1084, 8985, 8838,   15,  871, 4150,
        9270, 1061, 9440, 8290,  710, 4615, 4616, 1018, 7426, 2728, 4542,
        7278, 6979, 7731, 4782, 5133, 6660, 1499, 7569, 5267,  758, 4867,
        8826, 4085, 2948, 8382,  745, 8832, 3499, 2954, 4046, 3430,  650,
        8654]], dtype=uint64)

To check the result against the ground truth, run:

In [12]:
# For FLAT, the following will always be true
np.alltrue(result_i == ground_truth[query_id])

True

You can even run batches of searches, which are very efficiently implemented in TileDB-Vector-Search:

In [13]:
# Simply provide more than one query vectors
result_d, result_i = ivf_flat_index.query(np.array([query_vectors[5], query_vectors[6]]), nprobe=10, k=100)
result_i

array([[1097, 1239, 4943, 3227, 2607, 4443, 4246, 3112,  535, 4445, 2945,
        1707, 1896, 1626, 7132, 6767, 9690, 8666, 3699, 3460, 1532, 2329,
        8844, 2115,  660, 3652,  771, 1068, 9608, 2791, 9072, 4296, 9048,
        3506, 2753, 1445, 7837, 4051,  609, 9176, 4413, 3241, 3286, 7913,
        3614, 6039, 7669, 2378,  169, 5815, 3141, 1218, 3365,  703, 7322,
         121, 6585, 9999, 1863, 7214, 2571, 3065, 8867,  151,  226, 3649,
         172, 9083, 7348,  861, 1254, 3062,   96, 1834, 5995, 7169, 2381,
         754, 2629, 3769, 7504, 2706,  645, 3061,  806, 5323, 3391, 4324,
        3682, 4447, 3914, 5874, 2650, 9911, 9579, 1310, 6165, 4314,  667,
        7021],
       [2456, 3013, 1682, 8581, 2774, 3530,  924, 2732, 9701, 1916, 3687,
        1036, 4094, 9885, 2638, 8000, 1151, 1174, 2839, 3609, 2176, 9651,
        3996, 3943, 8284, 8642, 3249, 3954, 8517, 1468, 2107, 3854, 1323,
        3623, 8886, 8773, 3548, 3543,  694, 2435, 3298,  683, 2038, 1623,
        1038, 2702, 313

To query a vector search asset in a later session, you simply need to run the following command to initiate the index for queries:

In [14]:
index = vs.IVFFlatIndex(index_uri_ivf)
query_id = 77
result_d, result_i = index.query(np.array([query_vectors[query_id]]), nprobe=10, k=10)
result_i

array([[8578, 8275, 5332, 9153, 2092, 3290, 2010, 8004, 2949, 9190]],
      dtype=uint64)

# Cloud Object Store Support

TileDB natively supports several of the most widely-used Cloud object stores with no additional dependencies. For example, if you have configured your AWS S3 account with default credentials in `HOME/.aws`, then TileDB may be used to read and write directly from S3 with no additional configuration as follows:

```python
data_dir = #<where your source vector file resides>
output_uri = "s3://tiledb-isaiah2/vector_search/sift10k_flat"
index = vs.ingest(
    index_type = "IVF_FLAT",
    index_uri = output_uri,
    Ssource_uri = os.path.join(data_dir, "siftsmall_base.fvecs"),
    source_type = "FVEC",
    partitions = 100
)
```

For more information on TileDB’s cloud object storage support, see blog [TileDB 101: Cloud Object Storage](https://tiledb.com/blog/tiledb-101-cloud-object-storage). For additional configuration and authentication options for AWS S3, Azure, and GCS, see the following documentation:


* [TileDB Embedded Backends: S3](https://docs.tiledb.com/main/how-to/backends/s3)
* [TileDB Embedded Backends: Azure Blob Storage](https://docs.tiledb.com/main/how-to/backends/azure-blob-storage)
* [TileDB Embedded Backends: Google Cloud Storage](https://docs.tiledb.com/main/how-to/backends/gcs)

# Power Up with TileDB Cloud

If you wish to boost the vector search performance and enjoy some important data management features, you can use TileDB-Vector-Search on TileDB Cloud ([sign up](https://cloud.tiledb.com/auth/signup) and we will give you free credits for your trial). In this section I will describe how to perform serverless, distributed ingestion and vector search using TileDB Cloud’s task graphs. I will also cover the exciting data governance functionality of TileDB Cloud, which allows you to securely share your assets with other users, discover other users’ public work and log every action for auditing purposes.

If you are interested in delving into the general functionality of TileDB Cloud, you can read the following blog posts:

* [Getting Started with TileDB Cloud in &lt;4 Minutes](https://tiledb.com/blog/getting-started-with-tiledb-cloud-in-less-than-4-minutes)
* [TileDB Cloud: Notebooks](https://tiledb.com/blog/tiledb-cloud-notebooks)
* [TileDB Cloud: User-defined Functions](https://tiledb.com/blog/tiledb-cloud-user-defined-functions)
* [TileDB Cloud: Task Graphs](https://tiledb.com/blog/tiledb-cloud-task-graphs)

## Scaling with serverless, distributed task graphs

**Distributed ingestion** can greatly speed up and horizontally scale out the ingestion of vectors. In this example, we've ingested the SIFT 1 Billion vector dataset using the `IVF_FLAT` index (which involved computing K-means, a computationally intensive operation) in 46 minutes for a total cost of $11.385 in TileDB Cloud. 

To ingest in a distributed, parallel fashion, simply set the `mode` to "`BATCH`" for batch ingestion, and pass a `tiledb.cloud.Config()` parameter with your TileDB credentials:

```python
import tiledb
import tiledb.cloud
import tiledb.vector_search as vs

output_uri = "tiledb://TileDB-Inc/s3://tiledb-exaxmple/vector-search/ann_sift1b"
source_uri = "tiledb://TileDB-Inc/6a9a8e97-d99c-4ddb-829a-8455c794906e"

vs.ingest(    
    index_type = "IVF_FLAT",
    index_uri = output_uri,
    source_uri = source_uri,
    source_type = "TILEDB_ARRAY",
    partitions = 10_000,
    config = tiledb.cloud.Config(),
    mode = vs.Mode.BATCH)
```

Note that `ingest` automatically calculates the number of workers that will work in parallel to perform the ingestion, and there is no need to spin up any cluster - _everything is serverless!_

TileDB Cloud records all the details about the task graph, and lets you monitor it in real time.

[![image01-task-graph monitor.png](https://d2y9znfg8p5aha.cloudfront.net/cms/image01_task_graph_monitor_1ea74b888f.png)](https://d2y9znfg8p5aha.cloudfront.net/cms/image01_task_graph_monitor_1ea74b888f.png)

**Distributed queries** give you the capability of performing higher throughput queries with lower latency, yielding a much higher QPS. For example, in the example below we are able to submit a batch of 1000 query vectors to the 1 billion vector dataset for a cost of $0.10 in 23 seconds.  

Performing a distributed query is similar to the ingestion described above, but now you can set mode to `REALTIME` as it will be faster:

In [15]:
# If running locally, log in to TileDB Cloud
import os

tiledb.cloud.login(token=os.getenv("TILEDB_REST_TOKEN"))

In [16]:
uri = "tiledb://TileDB-Inc/ann_sift1b"
query_vector_uri = "tiledb://TileDB-Inc/bigann_1b_ground_truth_query"
n_vectors=10

index = vs.IVFFlatIndex(uri, config=tiledb.cloud.Config(), memory_budget=10)
query_vectors=np.transpose(vs.load_as_array(query_vector_uri, config=tiledb.cloud.Config()).astype(np.float32))[0:n_vectors]
result_d, result_i = index.query(query_vectors, k=10, nprobe=1, mode=tiledb.cloud.dag.Mode.REALTIME, num_partitions=20)
result_i[0:10]

array([[633840422, 683459400, 528266068, 109873774, 608153831, 825146167,
        762300501,  17551325, 630237822, 725616074],
       [335599671, 335599264, 335598977,  83311229, 682619471, 335599256,
        963017701, 335598760, 335599580, 335599807],
       [344643354, 246955639, 565542048, 751698684,  87364602, 840645909,
        550511822, 516688312, 668831661, 734216921],
       [186438378,  85244399, 384823375, 835264423, 446357863, 322140940,
        171195744, 857829688, 533547710, 463570876],
       [523076231,  83960482, 608028818, 487961680,  33024749,  85317553,
        136475748, 924522023, 684614270, 896442333],
       [786435621, 537081045, 152994050,  79006238, 280302530, 754967657,
        463050981, 999108052, 795662422, 739423679],
       [523602814, 937670694, 850429385, 207670705, 446595472, 158341966,
        122737998, 886060311, 567862264, 766553033],
       [121442525,  61428470, 704654443, 464551106, 107542667,  67712460,
        215032073, 392963508, 3931674

Vector search asset `tiledb://TileDB-Inc/ann_sift1b` is public (see “Governance” section below) and, thus, the above code will “just work” if you try to use it with your credentials.

Here is the task graph output of the above query.

[![image02-distributed-query-vector-search.png](https://d2y9znfg8p5aha.cloudfront.net/cms/image02_distributed_query_vector_search_553c476d66.png)](https://d2y9znfg8p5aha.cloudfront.net/cms/image02_distributed_query_vector_search_553c476d66.png)

## Governance

TileDB Cloud allows you to quickly explore any vector search datasets right in the UI. We have several public datasets available, including the original [ANN SIFT 1Billion Raw Vectors](https://cloud.tiledb.com/arrays/details/TileDB-Inc/8467cd03-c1e6-4c85-9593-96bf1a8a4c77/overview). This array lets you query the original vectors and get the data back as numpy arrays without having to parse the ivecs file. Simply slice directly by vector id!

In [17]:
uri = "tiledb://TileDB-Inc/ann_sift1b_raw_vectors"
with tiledb.open(uri, 'r', config=tiledb.cloud.Config()) as A:
  vector_id = 1
  print(A[vector_id])

OrderedDict([('values', array([129,   4,   6, 106,  34,   6,  89,   0,  34,  89,   6,  30,  77,
         6,  12, 127,   8,  76,  25,   3,   0,  14,  83,   7,  98,   6,
       127,  36,  17,  42,  35,  30,  43,  45,  92, 119,  51,   5,  43,
       119,  32,  14,  62,  18,   1,  41,  23,   2,  14,   1, 122,   6,
        39,  69,  90,  23,  67,  45,  17,  15,  21,  13,  41,   6,   0,
        10,  27,   6, 124,  27,   8,  23,  42,   8,   7,  19,  37,  33,
         0,  97,   4,   8,  68,  67,   1,   2,  33,  23,  25,  53,   2,
        10,  11,   0,   1,   3,  30,   0,  10,  32,   9,  69,  87,  12,
         8,  19,  45,  91,  37,  79,  61,  21,  22,   2,  26, 125,  30,
        42,  26,   0,   6,  82,   7, 120, 125,   6,   2,  14], dtype=uint8))])


Additional datasets include the pre-ingested BigANN dataset (with [`FLAT`](https://cloud.tiledb.com/vector-search/TileDB-Inc/9766cbed-e83f-4040-a715-82935d426962/overview) and [`IVF_FLAT`](https://cloud.tiledb.com/vector-search/TileDB-Inc/f4b551c3-fe2e-47a4-8625-28d98d92974f/overview) indexes), as well as the [tensorflow flowers](https://cloud.tiledb.com/vector-search/TileDB-Inc/a801f315-fddb-4ded-a23f-d1024457a3e3/overview) and [open drone map](https://cloud.tiledb.com/vector-search/TileDB-Inc/050b7de4-f93c-4c96-84b1-f74030ff9065/contents) datasets.

[![image03-public-vector-datasets.png](https://d2y9znfg8p5aha.cloudfront.net/cms/image03_public_vector_datasets_1a7e898d5e.png)](https://d2y9znfg8p5aha.cloudfront.net/cms/image03_public_vector_datasets_1a7e898d5e.png)

Let's take a look at the [BigANN SIFT 1 Billion dataset](https://cloud.tiledb.com/vector-search/TileDB-Inc/f4b551c3-fe2e-47a4-8625-28d98d92974f/overview). When you open up the dataset you can see the components, as well as the overview, sharing, and settings.

You can quickly share the dataset with any other user or organization on the sharing page.

[![image04-share-annsift1b.png](https://d2y9znfg8p5aha.cloudfront.net/cms/image04_share_annsift1b_9c6f435597.png)](https://d2y9znfg8p5aha.cloudfront.net/cms/image04_share_annsift1b_9c6f435597.png)

Any dataset can also easily be made public right on the setting page.

[![image05-makepublic-annsift1b.png](https://d2y9znfg8p5aha.cloudfront.net/cms/image05_makepublic_annsift1b_03fe5ef807.png)](https://d2y9znfg8p5aha.cloudfront.net/cms/image05_makepublic_annsift1b_03fe5ef807.png)

Logging access and providing a full audit trail is built directly into TileDB Cloud. When you are sharing data with third parties or making data public, it is important to be able to capture access and understand what people are doing with the data. TileDB provides logs for all access including specific details about the data access and code used to perform that access.


[![image06-logging-access-annsift1b.png](https://d2y9znfg8p5aha.cloudfront.net/cms/image06_logging_access_annsift1b_e8572f710c.png)](https://d2y9znfg8p5aha.cloudfront.net/cms/image06_logging_access_annsift1b_e8572f710c.png)


This gives you just a small taste of the power of flexibility of TileDB Cloud. Start your own exploration today and do not hesitate to [send us your feedback](https://tiledb.com/contact)!

# Stay Tuned

This notebook covered the very basics on TileDB’s vector search capabilities. Our team is working hard on multiple new exciting algorithms and features, so look out for the upcoming blogs on the TileDB-Vector-Search internals, benchmarks, integrations with LLMs, and more. You can also keep up with all other TileDB news by reading our [blog](https://tiledb.com/blog).